In [1]:
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_ollama import ChatOllama

In [2]:
llm = ChatOllama(model="qwen2.5", temperature=0)
mcp_client = MultiServerMCPClient(
    {
        'weater': {
            'url': 'http://127.0.0.1:3000/sse',
            'transport': 'sse',
        }
    }
)

In [3]:
import nest_asyncio
nest_asyncio.apply()

tools = await mcp_client.get_tools()

In [4]:
tools

[StructuredTool(name='get_weather', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x1074d53a0>)]

In [5]:
agent = create_react_agent(llm,tools)

In [6]:
from langchain_core.messages import HumanMessage, ToolMessage

In [18]:

async for chunk, metadata in agent.astream(
    {
        'messages': [
            HumanMessage(
                content="What is the weather in Singapore right now?"
            )
        ],
    },
    stream_mode='messages',
):
    print(chunk)

content='' additional_kwargs={} response_metadata={} id='run--d567a2cb-0f5e-404d-bc96-626ffaee4077' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Singapore'}, 'id': '03abbd80-0ea4-40b9-a64a-3314d5e69883', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'get_weather', 'args': '{"location": "Singapore"}', 'id': '03abbd80-0ea4-40b9-a64a-3314d5e69883', 'index': None, 'type': 'tool_call_chunk'}]
content='' additional_kwargs={} response_metadata={'model': 'qwen2.5', 'created_at': '2025-07-15T23:07:37.990948Z', 'done': True, 'done_reason': 'stop', 'total_duration': 486674584, 'load_duration': 38367625, 'prompt_eval_count': 155, 'prompt_eval_duration': 138783084, 'eval_count': 20, 'eval_duration': 308441125, 'model_name': 'qwen2.5'} id='run--d567a2cb-0f5e-404d-bc96-626ffaee4077' usage_metadata={'input_tokens': 155, 'output_tokens': 20, 'total_tokens': 175}
content='The weather in Singapore is sunny.' name='get_weather' id='ae9527b8-943f-461f-a51b-ae4dd9797b65' tool_call_id='03abbd

In [23]:
from langgraph.config import get_stream_writer

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
)

for update_type, chunk in agent.stream(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    stream_mode=["custom", "updates"]
):
    print(chunk)
    print("\n")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2025-07-15T23:09:04.548224Z', 'done': True, 'done_reason': 'stop', 'total_duration': 488774000, 'load_duration': 39312584, 'prompt_eval_count': 159, 'prompt_eval_duration': 135725458, 'eval_count': 20, 'eval_duration': 312555750, 'model_name': 'qwen2.5'}, id='run--06a60e87-0e5f-4b6a-888c-ecac99c3da74-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': '57decb07-217f-40f4-835c-87ccdbdf3a54', 'type': 'tool_call'}], usage_metadata={'input_tokens': 159, 'output_tokens': 20, 'total_tokens': 179})]}}


Looking up data for city: sf


{'tools': {'messages': [ToolMessage(content="It's always sunny in sf!", name='get_weather', tool_call_id='57decb07-217f-40f4-835c-87ccdbdf3a54')]}}


{'agent': {'messages': [AIMessage(content='The weather in San Francisco is always sunny! However, for a more accurate and detailed weather report, I can check the current temper